In [171]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Importing the Libraries

In [172]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

## Importing the Data

In [173]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

## Data Preprocessing

In [174]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [175]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


## Filling the missing values

In [176]:
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

In [177]:
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)

In [178]:
train_df = train_df.drop(['Cabin','Name','Ticket'], axis=1)
test_df = test_df.drop(['Cabin','Name','Ticket'], axis=1)

In [179]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [180]:
x_train = train_df.iloc[:,2:]
y_train = train_df.iloc[:,1]

## Encoding the categorical variables

In [181]:
x_train = pd.get_dummies(x_train)
test_df = pd.get_dummies(test_df)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Age         891 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   Sex_female  891 non-null    uint8  
 6   Sex_male    891 non-null    uint8  
 7   Embarked_C  891 non-null    uint8  
 8   Embarked_Q  891 non-null    uint8  
 9   Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(3), uint8(5)
memory usage: 39.3 KB


## Building and Tuning the Hyperparameters of the Random Forest model

In [182]:
rf = RandomForestClassifier()

In [183]:
score = cross_val_score(rf,x_train,y_train,cv=5,scoring='accuracy')
print(score.mean())

0.8103571652752495


In [184]:
n_estimators_rf = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features_rf = ['auto', 'sqrt']
max_depth_rf = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth_rf.append(None)
bootstrap_rf = [True, False]

random_grid_rf = {'n_estimators': n_estimators_rf,
               'max_features': max_features_rf,
               'max_depth': max_depth_rf,
               'bootstrap': bootstrap_rf}

print(random_grid_rf)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'bootstrap': [True, False]}


In [185]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid_rf, n_iter=100, cv=3, verbose=0, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42)

In [186]:
rf_random.best_params_

{'n_estimators': 200,
 'max_features': 'auto',
 'max_depth': 10,
 'bootstrap': True}

In [187]:
rf_cv = RandomForestClassifier(n_estimators=200, max_features='auto', max_depth=10, bootstrap=True)

In [188]:
score_cv = cross_val_score(rf_cv,x_train,y_train,cv=3,scoring='accuracy')
print(score_cv.mean())

0.8159371492704826


## Building and Tuning the Gradient Boosting Classifier

In [189]:
gbc = GradientBoostingClassifier(random_state=42)

In [190]:
print(cross_val_score(gbc,x_train,y_train,cv=10,scoring='accuracy').mean())

0.8328089887640449


In [191]:
loss_gbc = ['deviance','exponential']
learning_rate_gbc = [0.1,0.05,0.01]
n_estimators_gbc = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth_gbc = [int(x) for x in np.linspace(2, 5, num = 4)]
max_features_gbc = ['auto', 'sqrt', 'log2']

random_grid_gbc = {'loss':loss_gbc,
               'learning_rate':learning_rate_gbc,
               'n_estimators': n_estimators_gbc,
               'max_features': max_features_gbc,
               'max_depth': max_depth_gbc}

print(random_grid_gbc)

{'loss': ['deviance', 'exponential'], 'learning_rate': [0.1, 0.05, 0.01], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [2, 3, 4, 5]}


In [192]:
gbc_random = RandomizedSearchCV(estimator=gbc, param_distributions=random_grid_gbc, n_iter=100, cv=3, verbose=0, random_state=42, n_jobs=-1)
gbc_random.fit(x_train, y_train)

RandomizedSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.01],
                                        'loss': ['deviance', 'exponential'],
                                        'max_depth': [2, 3, 4, 5],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42)

In [193]:
gbc_random.best_params_

{'n_estimators': 1000,
 'max_features': 'log2',
 'max_depth': 5,
 'loss': 'exponential',
 'learning_rate': 0.01}

In [194]:
gbc_cv = GradientBoostingClassifier(n_estimators=1000,max_features='log2',max_depth=5,loss='exponential',learning_rate=0.01,random_state=42)

In [195]:
print(cross_val_score(gbc_cv,x_train,y_train,cv=3,scoring='accuracy').mean())

0.8215488215488217


## Predicting the results

In [196]:
gbc_cv.fit(x_train, y_train)
pred_gbc_cv = gbc_cv.predict(test_df.iloc[:,1:])

In [197]:
rf_cv.fit(x_train, y_train)
pred = rf_cv.predict(test_df.iloc[:,1:])

In [198]:
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': pred_gbc_cv})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
